In [ ]:
import os
import pandas as pd
import requests
from osgeo import ogr, osr
from datetime import datetime, timedelta

# Function to map tail numbers to ADS-B codes
def map_tail_number_to_adsb_code(tail_number):
    mapping = {
        "T-01": "a5cd6c", "T-02": "a5d123", "T-03": "a5d4da", "T-05": "ad66e3", "T-06": "adc0c4",
        "T-07": "aafd0b", "T-10": "a5c9b5", "T-101": "a2f996", "T-102": "a2fd4d", "T-103": "a30104",
        "T-104": "a304bb", "T-105": "a30872", "T-107": "a30fe0", "T-12": "a5d891", "T-131": "a07f21",
        "T-132": "a4c031", "T-133": "a4b50c", "T-137": "a0956b", "T-14": "a8215e", "T-142": "a19da0",
        "T-15": "a2b7fd", "T-152": "ac75f9", "T-16": "a5dfff", "T-160": "ab79cc", "T-161": "a3f3ac",
        "T-162": "a3f763", "T-163": "a42299", "T-164": "a442aa", "T-166": "c07cb2", "T-167": "a85052",
        "T-168": "a4ea6d", "T-169": "a380e6", "T-210": "a4b460", "T-260": "a47cbc", "T-261": "a48a3a",
        "T-262": "a47197", "T-263": "a46de0", "T-40": "A9A086", "T-41": "NA", "T-44": "c00e3e",
        "T-910": "a7f642", "T-911": "a11cbb", "T-912": "a69072", "T-914": "a7d27a", "T-944": "NA",
        "N130CG": "a07b6a"
    }
    return mapping.get(tail_number, "UNKNOWN_ADSB_CODE")

# Function to get OpenSky data
def get_opensky_data(start, end, bounds, icao24):
    url = 'https://opensky-network.org/api/states/all'
    params = {
        'begin': int(start.timestamp()),
        'end': int(end.timestamp()),
        #'icao24': icao24,
        'lamin': bounds[0],
        'lomin': bounds[1],
        'lamax': bounds[2],
        'lomax': bounds[3]
    }
    print(f"Requesting data with parameters: {params}")  # Debugging information
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        print(f"Response data: {data}")  # Debugging information
        return data['states'] if data['states'] else None
    else:
        print(f"Request failed with status code: {response.status_code}")  # Debugging information
        return None

# Function to process and save data
def process_data(data, output_csv, output_shp):
    if not data:
        return
    
    # Convert data to DataFrame
    columns = ['icao24', 'callsign', 'origin_country', 'time_position', 'last_contact',
               'longitude', 'latitude', 'baro_altitude', 'on_ground', 'velocity', 'true_track',
               'vertical_rate', 'sensors', 'geo_altitude', 'squawk', 'spi', 'position_source']
    df = pd.DataFrame(data, columns=columns)
    df = df.dropna(subset=['latitude', 'longitude'])

    if len(df) > 0:
        df.to_csv(output_csv, index=False)

        driver = ogr.GetDriverByName("ESRI Shapefile")
        data_src = driver.CreateDataSource(output_shp)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)  # 4326 = WGS84
        layer = data_src.CreateLayer(output_shp, srs, geom_type=ogr.wkbPoint)

        # Create attribute fields
        for col in columns:
            if df[col].dtype == 'float64':
                field_type = ogr.OFTReal
            else:
                field_type = ogr.OFTString
            field = ogr.FieldDefn(col, field_type)
            layer.CreateField(field)

        for _, row in df.iterrows():
            feature = ogr.Feature(layer.GetLayerDefn())
            for col in columns:
                feature.SetField(col, str(row[col]))

            # Create point geometry
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(float(row['longitude']), float(row['latitude']))
            feature.SetGeometry(point)
            layer.CreateFeature(feature)
            feature = None

        data_src = None

# Load the data
data = pd.read_csv(r"C:\Users\admin-magstadt\Desktop\ADSBflighttracking\ATU_2017_2021_1mile_ALL.txt", sep=",", header=0)

# Replace ':' with '_' in 'Drop_ID1' column
data['Drop_ID1'] = data['Drop_ID1'].str.replace(':', '_').str.replace('-', '_').str.replace(' ', '')

# List all filenames in the specified directory (without file extensions)
directory_path = r"C:\Users\admin-magstadt\Desktop\ADSBData\DropSamplesADSB"
filenames = [os.path.splitext(file)[0] for file in os.listdir(directory_path)]

# Check if modified 'Drop_ID1' values are in the list of filenames
data['FilenameMatch'] = data['Drop_ID1'].isin(filenames)

# Filter the DataFrame to only include rows where 'Drop_ID1' matches a filename
data = data[data['FilenameMatch']]

# Drop the 'FilenameMatch' column
data.drop(columns=['FilenameMatch'], inplace=True)

# Map tail numbers to ADSB codes
data["adsbCODE"] = data["Tail_Numbe"].apply(map_tail_number_to_adsb_code)

# Convert 'UTCdateTim' to datetime
data['UTCdateTim'] = pd.to_datetime(data['UTCdateTim'])

# Create time windows
time_change = timedelta(minutes=0.25)
data["Time2"] = data.UTCdateTim + time_change
data["Time1"] = data.UTCdateTim - time_change

time_change2 = timedelta(minutes=720)
data["Time2_2"] = data.UTCdateTim + time_change2
data["Time1_2"] = data.UTCdateTim - time_change2

# Define bounding box
data["Latitude1PLUS"] = data.Latitude1 + 1.02
data["Latitude1MINUS"] = data.Latitude1 - 1.02
data["Longitude1PLUS"] = data.Longitude1 + 1.02
data["Longitude1MINUS"] = data.Longitude1 - 1.02

dataLatitude1PLUS = data["Latitude1PLUS"].tolist()
dataLatitude1MINUS = data["Latitude1MINUS"].tolist()
dataLongitude1PLUS = data["Longitude1PLUS"].tolist()
dataLongitude1MINUS = data["Longitude1MINUS"].tolist()

datetime1 = data["Time1"].tolist()
datetime2 = data["Time2"].tolist()
datetime1_2 = data["Time1_2"].tolist()
datetime2_2 = data["Time2_2"].tolist()

data_DropID = data["Drop_ID1"].tolist()
dataADSBLIST = data["adsbCODE"].tolist()

# Convert datetime to UTC using tz_convert instead of tz_localize
datetime1_utc = [dt.tz_convert('UTC') for dt in datetime1]
datetime2_utc = [dt.tz_convert('UTC') for dt in datetime2]
datetime1_2_utc = [dt.tz_convert('UTC') for dt in datetime1_2]
datetime2_2_utc = [dt.tz_convert('UTC') for dt in datetime2_2]

OutputDirectoryDropSamples = r"C:\\Users\\admin-magstadt\\Desktop\\VLATfromJIM\\Output\\OutputDropSamples3\\"
OutputDirectoryNonDropSamples = r"C:\\Users\\admin-magstadt\\Desktop\\VLATfromJIM\\Output\\OutputNonDropSamples3\\"

print(len(data))

for i in range(len(data)):
    print(i)
    data_adsb = get_opensky_data(datetime1_utc[i], datetime2_utc[i],
                                 [dataLatitude1MINUS[i], dataLongitude1MINUS[i],
                                  dataLatitude1PLUS[i], dataLongitude1PLUS[i]],
                                 dataADSBLIST[i])
    print(data_adsb)  # Debugging information
    
    if data_adsb:
        output_csv = f"{OutputDirectoryDropSamples}{data_DropID[i]}.csv"
        output_shp = f"{OutputDirectoryDropSamples}{data_DropID[i]}.shp"
        process_data(data_adsb, output_csv, output_shp)
    
    data_adsb_long = get_opensky_data(datetime1_2_utc[i], datetime2_2_utc[i],
                                      [dataLatitude1MINUS[i], dataLongitude1MINUS[i],
                                       dataLatitude1PLUS[i], dataLongitude1PLUS[i]],
                                      dataADSBLIST[i])
    if data_adsb_long:
        output_csv = f"{OutputDirectoryNonDropSamples}{data_DropID[i]}.csv"
        output_shp = f"{OutputDirectoryNonDropSamples}{data_DropID[i]}.shp"
        process_data(data_adsb_long, output_csv, output_shp)

print("done")
